In [1]:
import time

In [2]:
import nanohub.remote as nr
from nanohub.uidl.ipywidgets import buildJSX
import pandas as pd
import numpy as np
import json, math, os, datetime, threading
import ipywidgets as w
from plotly.graph_objects import FigureWidget
from plotly.subplots import make_subplots
import types


start = datetime.datetime.now()
auth_data = { 'grant_type' : 'tool' }
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]  
    


In [3]:
session = nr.Session(auth_data)
SIM2L = "st4pnjunction"
SIM2L_R = 9
req_json = session.requestPost('dbexplorer/dbexplorer/tool_detail', data={'tool':SIM2L, 'simtool':True})
req_json = req_json.json()
params = req_json['results'][0][SIM2L]["input"]


In [4]:
def elementHandler (c):
    c['owner'].variant1 = "contained" if c['new'] == "Si" else "outlined"
    c['owner'].variant2 = "contained" if c['new'] == "Ge" else "outlined"
    c['owner'].variant3 = "contained" if c['new'] == "GaAs" else "outlined"
    c['owner'].variant4 = "contained" if c['new'] == "InP" else "outlined" 

Element = buildJSX('''
<Material.Paper value="{value:string(Si)}">
    <Material.ButtonGroup orientation="{orientation:string(vertical)}">
        <Material.Button variant="{variant1:string(contained)}" onClick="[stateChange(value,'Si')]">
            Si
        </Material.Button>
        <Material.Button variant="{variant2:string(outlined)}" onClick="[stateChange(value,'Ge')]">
            Ge
        </Material.Button>
        <Material.Button variant="{variant3:string(outlined)}" onClick="[stateChange(value,'GaAs')]">
            Ga-As
        </Material.Button>
        <Material.Button variant="{variant4:string(outlined)}" onClick="[stateChange(value,'InP')]">
            In-P
        </Material.Button>
    </Material.ButtonGroup>
</Material.Paper>
''')

Device = buildJSX('''
<Material.Paper _children="{children:array([])}" style="{style:object({})}"/>
''')

Button = buildJSX('''
<Material.Button style="{style:object({'width':'100%'})}" onClick="[propCall(onClick,true)]" variant="{variant:string(outlined)}">
    <Material.Typography gutterBottom="true" align="center" >
        {label:string(Next)}
    </Material.Typography>
</Material.Button>
''')

Region = buildJSX('''
<Material.Paper style="{style:object({'padding':'15px 20px 0px 20px'})}">
<Material.Typography gutterBottom="true" align="center" >
    {label:string( )}
</Material.Typography>
<Material.Slider 
    onChange="[stateChange(value,arguments[1])]"
    defaultValue="{value:integer(1)}"
    step="1"
    marks="{marks:array([])}"
    min="{min:integer(0)}"
    max="{max:integer(10)}"
    valueLabelDisplay="on"
/>
</Material.Paper>
''')

Voltage = buildJSX('''
<Material.Paper style="{style:object({'padding':'15px 20px 0px 20px'})}">
<Material.Typography gutterBottom="true" align="center" >
    {label:string( )}
</Material.Typography>
<Material.Slider 
    onChange="[stateChange(value,arguments[1])]"
    defaultValue="{value:number(0.5)}"
    step="{step:number(0.1)}"
    marks="{marks:array([])}"
    min="{min:number(0.0)}"
    max="{max:number(1.0)}"
    valueLabelDisplay="off"
/>
</Material.Paper>
''')

Output = buildJSX('''
<div style="{style:object({'padding':'15px 20px 15px 20px'})}">
    <Material.Select value="{value:string(IV Characteristic)}">
          <Material.MenuItem onClick="[stateChange(value,'IV Characteristic')]" value="IV Characteristic">IV CURVE</Material.MenuItem>
          <Material.MenuItem onClick="[stateChange(value,'CV Characteristic')]" value="CV Characteristic">CV CURVE</Material.MenuItem>
    </Material.Select>
</div>
''')


Draw = buildJSX('''
<Material.ButtonGroup orientation="horizontal" style="{style:object({'width':'100%'})}">
    <Material.Button style="{Pr:object({'width':'100%'})}">
        P
    </Material.Button>
    <Material.Button style="{Ir:object({'width':'100%'})}">
        I
    </Material.Button>
    <Material.Button style="{Nr:object({'width':'100%'})}">
        N
    </Material.Button>
</Material.ButtonGroup>
''')

Dialog = buildJSX('''
<Material.Dialog
      disableBackdropClick = ""
      disableEscapeKeyDown = ""
      maxWidth="xs"
      open="{isopen:boolean(false)}"
      
>
    <Material.DialogTitle>Submit Job</Material.DialogTitle>
    <Material.DialogContent dividers="true" >
        <Material.DialogContentText>
            To submit a new simulation, please configure the parameters.
        </Material.DialogContentText>
        <Material.FormControl style="{style:object({'width':'100%'})}">
            <Material.InputLabel>Na doping</Material.InputLabel>
            <Material.Select value="{Na:string(1e+15)}">
                <Material.MenuItem value="1e+15" onClick="[stateChange(Na,'1e+15')]">1e+15</Material.MenuItem>
                <Material.MenuItem value="1e+16" onClick="[stateChange(Na,'1e+16')]">1e+16</Material.MenuItem>
                <Material.MenuItem value="1e+17" onClick="[stateChange(Na,'1e+17')]">1e+17</Material.MenuItem>
                <Material.MenuItem value="1e+18" onClick="[stateChange(Na,'1e+18')]">1e+18</Material.MenuItem>
            </Material.Select>
        </Material.FormControl>
        <Material.FormControl style="{style:object({'width':'100%'})}">
            <Material.InputLabel>Nd doping</Material.InputLabel>
            <Material.Select value="{Nd:string(1e+15)}">
                <Material.MenuItem value="1e+15" onClick="[stateChange(Nd,'1e+15')]">1e+15</Material.MenuItem>
                <Material.MenuItem value="1e+16" onClick="[stateChange(Nd,'1e+16')]">1e+16</Material.MenuItem>
                <Material.MenuItem value="1e+17" onClick="[stateChange(Nd,'1e+17')]">1e+17</Material.MenuItem>
                <Material.MenuItem value="1e+18" onClick="[stateChange(Nd,'1e+18')]">1e+18</Material.MenuItem>
            </Material.Select>
        </Material.FormControl>
        <Material.DialogContentText>
            {status:string( )}
        </Material.DialogContentText>
    </Material.DialogContent>
    <Material.DialogActions>
        <Material.Button onClick="[stateChange(isopen,false)]">
            Close
        </Material.Button>
        <Material.Button onClick="[propCall(submitJob,'')]">
            Submit
        </Material.Button>
    </Material.DialogActions>
</Material.Dialog>
''')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
def updateLoading(stop):
    global v
    while True: 
        actual = datetime.datetime.now()
        delta1 = ((actual - start).total_seconds())%1/1
        delta2 = ((actual - start).total_seconds()+0.33)%1/1
        delta3 = ((actual - start).total_seconds()+0.66)%1/1
        fig.layout.shapes=[{
            'type': 'rect',
            'fillcolor': 'rgba(56,183,167, '+str(delta1)+')',
            'line': {
                'color': 'rgb(56,183,167)'
            },
            'yref' : 'paper',
            'xref' : 'paper',
            'x0' : 0.375,
            'x1' : 0.425,
            'y0' : 0.3,
            'y1' : 0.7,
        },{
            'type': 'rect',
            'fillcolor': 'rgba(56,183,167, '+str(delta2)+')',
            'line': {
                'color': 'rgb(56,183,167)'
            },
            'yref' : 'paper',
            'xref' : 'paper',
            'x0' : 0.475,
            'x1' : 0.525,
            'y0' : 0.3,
            'y1' : 0.7,
        },{
            'type': 'rect',
            'fillcolor': 'rgba(56,183,167, '+str(delta3)+')',
            'line': {
                'color': 'rgb(56,183,167)'
            },
            'yref' : 'paper',
            'xref' : 'paper',
            'x0' : 0.575,
            'x1' : 0.625,
            'y0' : 0.3,
            'y1' : 0.7,
        }]
        time.sleep(0.1)
        if stop(): 
            break
    fig.layout.shapes=[]
    fig.update_xaxes(matches='x', range=[0,v.value])
    fig.update_yaxes(matches='y')    


In [6]:
debug = w.Output()
detail = w.Textarea(
    value='',
    disabled=True,
    rows=4,
    layout=w.Layout(width="700px")
)
last_squid = ""
p = Region( value=3, marks= [{"value":0, "label":"0nm"},{"value":10, "label":"10nm"}], label="P-region", description="This is an Integer", layout=w.Layout(flex="1"))
i = Region(value=0, marks= [{"value":0, "label":"0nm"},{"value":10, "label":"10nm"}], label="I-region", description="This is an Integer", layout=w.Layout(flex="1"))
n = Region(value=6, marks= [{"value":0, "label":"0nm"},{"value":10, "label":"10nm"}], label="N-region", description="This is an Integer", layout=w.Layout(flex="1"))
v = Voltage(value=0.6, min=0.2, max = 1.2, marks= [{"value":0.4, "label":"0.4V"},{"value":0.6, "label":"0.6V"},{"value":0.8, "label":"0.8V"},{"value":1.0, "label":"1.0V"}], label="Voltage", description="Aplied Voltage", layout=w.Layout(flex="1"))
p.style ={'backgroundColor': 'rgba(255,0,0,0.2)', 'padding' : '15px 30px 5px 20px'}
i.style ={'backgroundColor': 'rgba(255,255,255,0.2)', 'padding' : '15px 30px 5px 30px'}
n.style ={'backgroundColor': 'rgba(0,0,255,0.2)', 'padding' : '15px 30px 5px 30px'}
e = Element(value = "Si", label="Select Material", orientation="horizontal")
b = Button(label="Add Point")
o = Output()
dtotal = p.value + i.value + n.value
dr = Draw(
    Pr={'backgroundColor': 'rgba(255,0,0,0.2)', 'width':str((p.value/dtotal)*100)+"%"},
    Ir={'backgroundColor': 'rgba(255,255,255,0.2)', 'width':str((i.value/dtotal)*100)+"%"},
    Nr={'backgroundColor': 'rgba(0,0,255,0.2)', 'width':str((n.value/dtotal)*100)+"%"}
)
d = Device(children=[dr,v,o,e,p,i,n,b],layout=w.Layout(flex="1"))
s=Dialog(isopen=False)


In [7]:
def openDialog (*args, **kwargs):
    s.isopen=True

def SubmitJob( *kargs, **kwargs ):  
    with debug:
        #print (kwargs)
        s.status = "Submitting the job"
        data = {
          'name': SIM2L,
          'revision': SIM2L_R,
          'inputs': {
            'p_len' : p.value,
            'p_node' : p.value*20,
            'i_len' : i.value,
            'i_node' : i.value*20,
            'n_len' : n.value,
            'n_node' : n.value*20,
            'materialp' : e.value,
            'Na' : float(s.Na),
            'Nd' : float(s.Nd)
          },
          'outputs': [o.value]
        }
        try :            
            status = "submitted"
            req_json = session.requestPost('dbexplorer/simtools/run', data=json.dumps(data))
            res = req_json.json()
            request_id = res['id']
            print(req_json)
            req_json = req_json.json()
            s.status = "Job allocated with id " + str(request_id)
            status = {'finished' : False}
            while ( status != "INDEXED" and status != "INVALID"):
                time.sleep(2)
                req_json = session.requestPost('dbexplorer/simtools/run/' + str(request_id), timeout=20)
                if req_json.status_code >= 400:
                    raise Exception( "Error" + req_json.status_code)     
                res = req_json.json()
                if "status" in res:
                    status = res["status"]
                    s.status = "Job " + str(request_id) + " " + str(res["status"]) + " - " + str(res["message"])
            if status == "INDEXED":
                if "outputs" in res and o.value in res["outputs"]:
                    new_x = res["outputs"][o.value]["voltage"]
                    new_y = res["outputs"][o.value]["function"]
                    na = np.arange(14.5,19.5,1)
                    nd = np.arange(14.5,19.5,1)
                    r = np.argwhere(na-math.log10(data['inputs']['Na'])>=0)
                    c = np.argwhere(nd-math.log10(data['inputs']['Nd'])>=0)
                    if len(r) > 0 and len(c) > 0:
                        r=r[0]
                        c=c[0]
                        if len(r) > 0 and len(c) > 0:
                            r=r[0]
                            c=c[0]
                            fig.add_trace({
                                'type': 'scatter',
                                #'mode':'markers',
                                'mode':'lines',
                                'name' :  "",#res['squid'],
                                'hovertemplate':'Voltage: %{x}<BR>Current: %{y}<BR>' + 'NA:'+"{:.1e}".format(data['inputs']['Na'])+'<BR>ND:'+"{:.1e}".format(data['inputs']['Nd']),
                                'showlegend' : False,
                                'marker' : {
                                    'color':'lightgreen',
                                    'size':10,
                                    'line':{
                                        'color':'MediumPurple',
                                        'width':1
                                    }
                                },
                                'x':[x for x in new_x if x < v.value],
                                'y':[new_y[i] for i,x in enumerate(new_x) if x < v.value],
                                }, row=r, col=c
                            )              
            #print(params)
            s.status = ""
            s.isopen = False
        except Exception as error:
            print(error)
            s.status = "There has been an error, try later\n"+str(error)
            s.isopen = True
        except OSError as error:
            s.status = "There has been an error, try later\n"+str(error)
            s.isopen = True
        except:
            s.status = "There has been an error, try later\n"
            s.isopen = True



In [8]:

BINS = 4
ctitles = ["Nd: (" + str(i) + "-" + str(i+1) + ")" for i in np.arange(14.5,20.5,1)]
rtitles = ["Na: (" + str(i) + "-" + str(i+1) + ")" for i in np.arange(14.5,20.5,1)]
fig = FigureWidget(make_subplots(rows=BINS,cols=BINS,shared_xaxes=True,shared_yaxes=True,vertical_spacing=0.01,horizontal_spacing=0.05, column_titles=ctitles, row_titles=rtitles))
fig.layout.width = 700
fig.layout.height = 600
fig.layout.margin = {'l':10,'r':15,'b':1,'t':45} 
fig.update_xaxes(matches='x', range=[0,1.0])
fig.update_yaxes(matches='y')


def UpdatePlot( update ):
    th = threading.Thread(target=updateLoading)
    stop_threads = False
    t1 = threading.Thread(target = updateLoading, args =(lambda : stop_threads, )) 
    t1.start() 
    global fig, debug, SIM2L
    #updateLoading(True)
    #debug.clear_output()
    with debug:
        #print (update)
        fig.data = []
        VOLTAGE = [0,v.value*2]
        TAUN = [(1e-10),(1e-10)]
        TAUP = [(1e-10),(1e-10)]
        P_LEN = [p.value,p.value]
        N_LEN = [n.value,n.value]
        TEMPERATURE = [300,300]
        I_LEN = [i.value,i.value]
        MATERIALP = e.value
        OUTPUT = 'output.' + o.value
        IMPURITY = "false"
        TOOL = SIM2L
        search = {
            'tool':TOOL, 
            'simtool':True, 
            'filters':json.dumps([        
                {'field':'input.p_len','value':P_LEN[0],'operation':'>='},
                {'field':'input.p_len','value':P_LEN[1],'operation':'<='},
                {'field':'input.n_len','value':N_LEN[0],'operation':'>='},
                {'field':'input.n_len','value':N_LEN[1],'operation':'<='},
                {'field':'input.vsweep_high','value':0,'operation':'>'},
                {'field':'input.taun','value':TAUN[0],'operation':'>='},
                {'field':'input.taun','value':TAUN[1],'operation':'<='},
                {'field':'input.taup','value':TAUP[0],'operation':'>='},
                {'field':'input.taup','value':TAUP[1],'operation':'<='},
                {'field':'input.temperature','value':TEMPERATURE[0],'operation':'>='},
                {'field':'input.temperature','value':TEMPERATURE[1],'operation':'<='},
                {'field':'input.i_len','value':I_LEN[0],'operation':'>='},
                {'field':'input.i_len','value':I_LEN[1],'operation':'<='},
                {'field':'input.materialp','value':MATERIALP,'operation':'=='},
                {'field':'input.impurity','value':IMPURITY,'operation':'=='},
                {'field':'input.Nd','value':5e14,'operation':'>'},
                {'field':'input.Na','value':5e14,'operation':'>'},
                {'field':'input.Nd','value':5e18,'operation':'<'},
                {'field':'input.Na','value':5e18,'operation':'<'},
                {'field':'input.vsweep_high','value':VOLTAGE[0],'operation':'>='},
                {'field':'input.vsweep_high','value':VOLTAGE[1],'operation':'<='},
                
            ]),
            'results':json.dumps([
                'input.Na', 
                'input.Nd',
                'input.vsweep_high',
                OUTPUT
            ]),    
            #'random':1,   
            'limit':50, 
            'revision':0
        }
        req_json = session.requestPost('dbexplorer/dbexplorer/search', data=search, timeout=300)
        req_json = req_json.json()
        #df = pd.DataFrame(req_json['results'])
        #display(df)
        #h,na,nd = np.histogram2d(np.log10(df['input.Na']), np.log10(df['input.Nd']), bins=BINS-1)
        na = np.arange(14.5,19.5,1)
        nd = np.arange(14.5,19.5,1)
        cols = []
        rows = []
        traces = []
        events = []
        for res in req_json['results']:
            r = np.argwhere(na-math.log10(res['input.Na'])>=0)
            c = np.argwhere(nd-math.log10(res['input.Nd'])>=0)
            #print (math.log10(res['input.Na']), math.log10(res['input.Nd']))
            #print (r,c, len(r), len(c))
            if len(r) > 0 and len(c) > 0:
                r=r[0]
                c=c[0]
                if len(r) > 0 and len(c) > 0:
                    #print (r,c)
                    r=r[0]
                    c=c[0]
                    #print (r,c)
                    if OUTPUT in res and res[OUTPUT]['voltage'] is not None and res[OUTPUT]['function'] is not None :
                        rows.append(r)
                        cols.append(c)
                        xn = [x for x in res[OUTPUT]['voltage'] if x < v.value]
                        yn = [res[OUTPUT]['function'][i] for i,x in enumerate(res[OUTPUT]['voltage']) if x < v.value]
                        traces.append({
                            'type': 'scatter',
                            #'mode':'markers',
                            'mode':'markers+lines',
                            'name' :  "",#res['squid'],
                            'hovertemplate':'Voltage: %{x}<BR>Current: %{y}<BR>' + 'NA:'+"{:.1e}".format(res['input.Na'])+'<BR>ND:'+"{:.1e}".format(res['input.Nd'])+'<BR>' + res['squid'],
                            'showlegend' : False,
                            'marker' : {
                                'color':'LightSkyBlue',
                                'size':2,
                                'line':{
                                    'color':'MediumPurple',
                                    'width':1
                                }
                            },
                            'x':xn,
                            'y':yn,
                        })
        fig.add_traces(traces, rows=rows, cols=cols)         
    stop_threads = True
    t1.join() 
e.unobserve_all()    
p.unobserve_all()    
n.unobserve_all()    
i.unobserve_all()    
o.unobserve_all()    


e.observe(elementHandler,  names="value")
e.observe(UpdatePlot, names="value") 
p.observe(UpdatePlot, names="value") 
n.observe(UpdatePlot, names="value")    
i.observe(UpdatePlot, names="value")    
o.observe(UpdatePlot, names="value")    
v.observe(UpdatePlot, names="value")    
b.onClick(openDialog)
s.submitJob(SubmitJob)


In [9]:
display(w.HTML("<style>html {font-size:16px} </style>"))
display(w.HTML("<style>div.output_subarea {padding:0px} </style>"))
display(s)
display(w.HBox([w.VBox([d]),w.VBox([fig,detail], layout=w.Layout(border="1px solid #BBB"))]))
display(debug)
UpdatePlot( None )

HTML(value='<style>html {font-size:16px} </style>')

HTML(value='<style>div.output_subarea {padding:0px} </style>')

UIDLa563b75e5e6848a8956d12b36b5b4662Widget(style={'width': '100%'})

Output()

In [10]:
len(fig.data)

17